In [0]:
import os
os.chdir("/content/drive/My Drive/Pneumonia")


In [0]:
import numpy as np
import pandas as pd
from numpy import load
train = load("train_img.npy")
train_labels = load("train_labels.npy")
test = load("test_img.npy")
test_labels = load("test_labels.npy")


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2

from keras.models import Sequential, Model
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical


from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization, Convolution2D , MaxPooling2D, Activation

In [0]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
train_enc = ohe.fit_transform(train_labels.reshape(-1,1))
test_enc = ohe.transform(test_labels.reshape(-1,1))

In [0]:
train_enc = train_enc.toarray()
test_enc = test_enc.toarray()

In [15]:
from keras.applications.vgg16 import VGG16
model = VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3))

58892288/58889256 [==============================] - 2s 0us/step


In [16]:
for layer in model.layers[:-2]:
  layer.trainable = False
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dense(1024, activation = "relu")(x)
predictions = Dense(3, activation="softmax")(x)
tl_model = Model(input = model.input ,  output = predictions)
tl_model.compile(Adam(), loss = "categorical_crossentropy", metrics=['accuracy'] )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [0]:
gen = ImageDataGenerator(
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True    
)
batches=gen.flow(train,train_enc,batch_size = 16, shuffle = True)
valbatches = gen.flow(test, test_enc, batch_size = 16, shuffle = True)




In [24]:
history=tl_model.fit_generator(generator=batches, steps_per_epoch=batches.n//batches.batch_size, epochs=10, validation_data=valbatches, validation_steps=valbatches.n//valbatches.batch_size,callbacks=[EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto') ,ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)])

Epoch 1/10
326/326 [==============================] - 87s 267ms/step - loss: 0.4715 - accuracy: 0.7962 - val_loss: 0.3931 - val_accuracy: 0.7949
Epoch 2/10
326/326 [==============================] - 86s 262ms/step - loss: 0.4780 - accuracy: 0.7995 - val_loss: 1.1863 - val_accuracy: 0.7965
Epoch 3/10
326/326 [==============================] - 84s 259ms/step - loss: 0.4591 - accuracy: 0.8039 - val_loss: 1.1669 - val_accuracy: 0.7788
Epoch 4/10
326/326 [==============================] - 85s 262ms/step - loss: 0.4455 - accuracy: 0.8089 - val_loss: 0.2703 - val_accuracy: 0.7853
Epoch 5/10
326/326 [==============================] - 86s 264ms/step - loss: 0.4589 - accuracy: 0.8037 - val_loss: 0.5207 - val_accuracy: 0.8045
Epoch 6/10
326/326 [==============================] - 85s 262ms/step - loss: 0.4335 - accuracy: 0.8135 - val_loss: 0.3813 - val_accuracy: 0.8221
Epoch 7/10
326/326 [==============================] - 85s 262ms/step - loss: 0.4522 - accuracy: 0.8041 - val_loss: 1.1524 - val_ac